In [4]:
%matplotlib widget
from util.sys import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import (
    generate_nx_graph,
    transform_list,
    generate_skeleton,
    generate_nx_graph_from_skeleton,
    from_connection_tab,
)
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime, timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
import os
from experiment_class import Experiment
import cv2
from IPython.display import clear_output
from random import choice
from sklearn import datasets, linear_model

In [5]:
plate = 13
directory = (
    "//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE"
)
listdir = os.listdir(directory)
list_dir_interest = [
    name
    for name in listdir
    if name.split("_")[-1] == f'Plate{0 if plate<10 else ""}{plate}'
]
ss = [name.split("_")[0] for name in list_dir_interest]
ff = [name.split("_")[1] for name in list_dir_interest]
dates_datetime = [
    datetime(
        year=int(ss[i][:4]),
        month=int(ss[i][4:6]),
        day=int(ss[i][6:8]),
        hour=int(ff[i][0:2]),
        minute=int(ff[i][2:4]),
    )
    for i in range(len(list_dir_interest))
]
dates_datetime.sort()
dates_datetime_chosen = dates_datetime[1:25]
dates = [
    f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}'
    for date in dates_datetime_chosen
]

In [6]:
def pinpoint_anastomosis(expe, t):
    nx_graph_tm1 = expe.nx_graph[t]
    nx_grapht = expe.nx_graph[t + 1]
    from_tip = expe.connections[t]
    pos_tm1 = expe.positions[t]
    anastomosis = []
    origins = []
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node) == 1]
    number_anastomosis = 0

    def dist_branch(node, nx_graph, pos):
        mini = np.inf
        for edge in nx_graph.edges:
            pixel_list = nx_graph.get_edge_data(*edge)["pixel_list"]
            if np.linalg.norm(np.array(pixel_list[0]) - np.array(pos[node])) <= 5000:
                distance = np.min(
                    np.linalg.norm(np.array(pixel_list) - np.array(pos[node]), axis=1)
                )
                if distance < mini:
                    mini = distance
        return mini

    def count_neighbors_is_from_root(equ_list, nx_graph, root):
        count = 0
        for neighbor in nx_graph.neighbors(root):
            if neighbor in equ_list:
                count += 1
        return count

    for tip in tips:
        #         print(tip)
        consequence = from_tip[tip]
        for node in consequence:
            if (
                node in nx_grapht.nodes
                and nx_grapht.degree(node) >= 3
                and count_neighbors_is_from_root(consequence, nx_grapht, node) < 2
            ):
                #                 if node==2753:
                #                     print(count_neighbors_is_from_root(consequence,nx_grapht,node))
                #                     print(list(nx_grapht.neighbors(node)))
                anastomosis.append(node)
                origins.append(tip)
                number_anastomosis += 1
        if (
            tip not in nx_grapht.nodes
            and dist_branch(tip, nx_grapht, pos_tm1) <= 30
            and nx_graph_tm1.get_edge_data(*list(nx_graph_tm1.edges(tip))[0])["weight"]
            >= 20
        ):
            origins.append(tip)
            number_anastomosis += 1 / 2
    return (anastomosis, origins, number_anastomosis)

In [7]:
exp = Experiment(13)
exp.load(dates, local=True, raw=False)

In [10]:
exp = pickle.load(open(f"Data/" + f"experiment_{13}.pick", "rb"))

In [259]:
def clean_exp(experiment):
    ts = {}
    for node in experiment.nodes:
        ts[node.label] = []
        for t in range(len(experiment.nx_graph)):
            if node.is_in(t):
                ts[node.label].append(t)
    to_remove = []
    for node in experiment.nodes:
        posit = node.pos(ts[node.label][0])
        if (
            len(ts[node.label]) == 1
            and ts[node.label][0] != len(experiment.nx_graph) - 1
            and posit[0] < 19000
            and posit[1] > 2000
        ):
            to_remove.append(node.label)
    return (to_remove, ts)

In [260]:
to_look, tis = clean_exp(exp)

In [261]:
len(to_look)

4446

In [177]:
np.mean(number_tips)

3.766

In [251]:
exp.get_node(node).pos(1)

array([18855,   885])

In [271]:
node = choice(to_look)

In [272]:
exp.plot(
    [tis[node][0], tis[node][0] + 1, tis[node][0] + 2], node_lists=[[node]], compress=5
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [268]:
plt.close("all")

In [215]:
exp.plot([18, 19, 20], node_lists=[[11698]] * 3, compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [212]:
for i in range(24):
    if exp.get_node(11698).is_in(i):
        print(i)

18
19
20
21
22


In [213]:
exp.get_node(11698).pos(18)

array([9357, 7821])

In [7]:
tip_track = {}
tip_ts = {}
for node in exp.nodes:
    for t in range(len(exp.dates)):
        if node.is_in(t) and node.degree(t) == 1:
            track = []
            for tsup in range(t, len(exp.dates)):
                if node.is_in(tsup):
                    track.append(node.pos(tsup))
            tip_track[node.label] = track
            tip_ts[node.label] = t
            break
tip_growth = {
    tip: [
        np.linalg.norm(tip_track[tip][i + 1] - tip_track[tip][i])
        for i in range(len(tip_track[tip]) - 1)
    ]
    for tip in tip_track.keys()
}
tip_of_interest = [
    tip for tip in tip_growth.keys() if np.any(np.array(tip_growth[tip]) >= 100)
]

In [8]:
tip = choice(tip_of_interest)
node_interest = tip
poss = exp.positions
ts = [t for t in range(len(poss)) if exp.get_node(tip).is_in(t)]

In [9]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("time (h)")
ax.set_ylabel("growth (mm/day)")
for i in range(20):
    tip = choice(tip_of_interest)
    (spotting,) = ax.plot(
        np.array(range(len(tip_growth[tip]))) * 4,
        np.array(tip_growth[tip]) * 1.725 / 4 * 24 / 1000,
    )
    spotting.set_label(str(tip))
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
def get_angle(pos1, pos2):
    vector = pos2 - pos1
    unit_vector = vector / np.linalg.norm(vector)
    vertical_vector = np.array([-1, 0])
    dot_product = np.dot(vertical_vector, unit_vector)
    if (
        vertical_vector[1] * vector[0] - vertical_vector[0] * vector[1] >= 0
    ):  # determinant
        angle = np.arccos(dot_product) / (2 * np.pi) * 360
    else:
        angle = -np.arccos(dot_product) / (2 * np.pi) * 360
    return angle


def becomes_degree3(node):
    node_obj = exp.get_node(node)
    for t in range(len(exp.nx_graph)):
        if node_obj.is_in(t) and node_obj.degree(t) == 3:
            return (True, t)
    return (False, None)

In [21]:
min_growth = 30
anastomose = [tip for tip in tip_of_interest if becomes_degree3(tip)[0]]
stop = [tip for tip in tip_of_interest if not becomes_degree3(tip)[0]]
stop_with_long_growth = [tip for tip in stop if np.sum(tip_growth[tip]) > 500]
anastomose_with_long_growth = [
    tip for tip in anastomose if np.sum(tip_growth[tip]) > 500
]
angles = {
    tip: [
        get_angle(tip_track[tip][i + 1], tip_track[tip][i])
        for i in range(len(tip_track[tip]) - 1)
        if tip_growth[tip][i] >= min_growth
    ]
    for tip in stop
}
angles_last = {
    tipi: [angles[tipi][i + 1] - angles[tipi][i] for i in range(len(angles[tipi]) - 1)]
    for tipi in angles.keys()
}
tis = {
    tip: [i for i in range(len(tip_track[tip]) - 1) if tip_growth[tip][i] >= min_growth]
    for tip in stop
}
tips_for_angle = [
    tip
    for tip in stop
    if len(angles[tip]) >= 4 and not np.any(np.abs(np.array(angles_last[tip])) >= 90)
]
distances = {
    tip: [
        np.linalg.norm(tip_track[tip][t] - tip_track[tip][tis[tip][0]])
        for t in tis[tip]
    ]
    for tip in tips_for_angle
}

In [16]:
len(tips_for_angle)

55

In [63]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("time (h)")
ax.set_ylabel("angle_from_begining(°)")
for tip in tips_for_angle[0:10]:
    if len(angles[tip]) >= 1 and not np.any(np.abs(np.array(angles_last[tip])) >= 90):
        (spotting,) = ax.plot(
            np.array(tis[tip]) * 4,
            (np.array(angles[tip]) - angles[tip][0] + 180) % 360 - 180,
        )
        spotting.set_label(str(tip))
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
plt.close("all")

In [64]:
tip = 79
node_interest = tip
poss = exp.positions
ts = [t for t in range(len(poss)) if exp.get_node(tip).is_in(t)]
pos_problem = [poss[i][node_interest] for i in ts]
xbegin = [pos_problem[i][0] - 1500 for i in range(len(ts))]
ybegin = [pos_problem[i][1] - 1500 for i in range(len(ts))]
xend = [pos_problem[i][0] + 1500 for i in range(len(ts))]
yend = [pos_problem[i][1] + 1500 for i in range(len(ts))]
skeletons_small = []
for i in range(len(ts)):
    skeletons_small.append(
        exp.skeletons[ts[i]][xbegin[i] : xend[i], ybegin[i] : yend[i]]
    )
if ts[-1] < len(exp.skeletons) - 1:
    skeletons_small.append(
        exp.skeletons[ts[-1] + 1][xbegin[-1] : xend[-1], ybegin[-1] : yend[-1]]
    )
node_smalls = []
for i in range(len(ts)):
    node_smalls.append(
        [
            node
            for node in exp.nx_graph[ts[i]].nodes
            if (
                xbegin[i] < poss[ts[i]][node][0] < xend[i]
                and ybegin[i] < poss[ts[i]][node][1] < yend[i]
                and exp.nx_graph[ts[i]].degree(node) >= 1
            )
        ]
    )
kernel = np.ones((5, 5), np.uint8)
skeletons_small_dilated = [
    cv2.dilate(skeleton.todense().astype(np.uint8), kernel, iterations=1)
    for skeleton in skeletons_small
]
for tp1 in range(len(node_smalls)):
    plot_t_tp1(
        [node_interest],
        [node_interest],
        poss[ts[tp1]],
        poss[ts[tp1]],
        skeletons_small_dilated[tp1],
        skeletons_small_dilated[tp1],
        shift=(xbegin[tp1], ybegin[tp1]),
        save=f"Data/video_test/network_timestep_{tp1}",
        time=f"t={4*tp1}h",
    )
    clear_output(wait=True)
if ts[-1] < len(exp.skeletons) - 1:
    plot_t_tp1(
        [],
        [],
        poss[ts[-1] + 1],
        poss[ts[-1] + 1],
        skeletons_small_dilated[-1],
        skeletons_small_dilated[-1],
        shift=(xbegin[-1], ybegin[-1]),
        save=f"Data/video_test/network_timestep_{len(node_smalls)}",
        time=f"t={len(node_smalls)}h",
    )
    clear_output(wait=True)
images = []
for t in range(len(node_smalls)):
    images.append(imageio.imread(f"Data/video_test/network_timestep_{t}.png"))
if ts[-1] < len(exp.skeletons) - 1:
    images.append(
        imageio.imread(f"Data/video_test/network_timestep_{len(node_smalls)}.png")
    )
imageio.mimsave(
    f"Data/video_test/tips_for_angle/{node_interest}movie_track.gif", images, duration=1
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("distance (mm)")
ax.set_ylabel("angle_from_begining(°)")
for tip in [tip]:
    if len(angles[tip]) >= 1 and not np.any(np.abs(np.array(angles_last[tip])) >= 90):
        (spotting,) = ax.plot(
            np.array(distances[tip]) * 1.725 / 1000 + 1,
            (np.array(angles[tip]) - angles[tip][0] + 180) % 360 - 180,
        )
#         spotting.set_label(str(tip))
# ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
angles_interest = (
    ((np.array(angles[tip]) - angles[tip][0] + 180) % 360 - 180) / 360 * 2 * np.pi
)
distances_interest = np.array(distances[tip]) * 1.725 / 1000 + 1
dist = []
cosins = []
for i, angle in enumerate(angles_interest):
    for j in range(i + 1, len(angles_interest)):
        dist.append(distances_interest[j] - distances_interest[i])
        cosins.append(np.cos(angles_interest[j] - angles_interest[i]))
num_slices = len(distances[tip]) // 2
slices = [
    (i * max(dist) / num_slices, (i + 1) * max(dist) / num_slices)
    for i in range(num_slices)
]
end_slice = [slico[1] for slico in slices]
slice_cosins = [
    [cos for k, cos in enumerate(cosins) if slico[0] <= dist[k] < slico[1]]
    for slico in slices
]
slice_cosins_av = [np.mean(slice_cos) for slice_cos in slice_cosins]
slice_size = [len(slice_cos) for slice_cos in slice_cosins]
regr = linear_model.LinearRegression()
regr.fit(
    np.array(end_slice).reshape(-1, 1),
    np.log(np.array(slice_cosins_av)).reshape(-1, 1),
    sample_weight=slice_size,
)
fit = regr.predict(np.array(end_slice).reshape(-1, 1))
print(-1 / regr.coef_[0][0])
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("distance (mm)")
ax.set_ylabel("angle_from_begining(log(°))")
ax.scatter(end_slice, np.log(np.array(slice_cosins_av)))
plt.plot(end_slice, fit)
#         spotting.set_label(str(tip))
# ax.legend()
plt.show()

3165.5831279573613


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
persistence_lenghts = {}
for tip in tips_for_angle:
    angles_interest = (
        ((np.array(angles[tip]) - angles[tip][0] + 180) % 360 - 180) / 360 * 2 * np.pi
    )
    distances_interest = np.array(distances[tip]) * 1.725 / 1000 + 1
    dist = []
    cosins = []
    for i, angle in enumerate(angles_interest):
        for j in range(i + 1, len(angles_interest)):
            dist.append(distances_interest[j] - distances_interest[i])
            cosins.append(np.cos(angles_interest[j] - angles_interest[i]))
    num_slices = len(distances[tip]) // 2
    slices = [
        (i * max(dist) / num_slices, (i + 1) * max(dist) / num_slices)
        for i in range(num_slices)
    ]
    end_slice = [slico[1] for slico in slices]
    slice_cosins = [
        [cos for k, cos in enumerate(cosins) if slico[0] <= dist[k] < slico[1]]
        for slico in slices
    ]
    slice_cosins_av = [np.mean(slice_cos) for slice_cos in slice_cosins]
    slice_size = [len(slice_cos) for slice_cos in slice_cosins]
    regr = linear_model.LinearRegression()
    regr.fit(
        np.array(end_slice).reshape(-1, 1),
        np.log(np.array(slice_cosins_av)).reshape(-1, 1),
        sample_weight=slice_size,
    )
    fit = regr.predict(np.array(end_slice).reshape(-1, 1))
    if regr.coef_[0][0] < 0:
        persistence_lenghts[tip] = -1 / regr.coef_[0][0]

In [ ]:
# persistence_lenghts.remove(np.max(persistence_lenghts))
# persistence_lenghts.remove(np.min(persistence_lenghts))
persistence_lenghts

In [68]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_ylabel("Number in this category")
ax.set_xlabel("log10(persistence_length) (log(mm))")
ax.hist(np.log10(list(persistence_lenghts.values())))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 3.,  7., 17.,  6.,  6.,  4.,  0.,  1.,  0.,  1.]),
 array([-0.13519238,  0.33862752,  0.81244741,  1.28626731,  1.7600872 ,
         2.23390709,  2.70772699,  3.18154688,  3.65536678,  4.12918667,
         4.60300656]),
 <BarContainer object of 10 artists>)

In [69]:
plt.close("all")

In [70]:
for i in range(10):
    tip = choice(anastomose)
    node_interest = tip
    poss = exp.positions
    ts = [t for t in range(len(poss)) if exp.get_node(tip).is_in(t)]
    pos_problem = [poss[i][node_interest] for i in ts]
    xbegin = [pos_problem[i][0] - 1500 for i in range(len(ts))]
    ybegin = [pos_problem[i][1] - 1500 for i in range(len(ts))]
    xend = [pos_problem[i][0] + 1500 for i in range(len(ts))]
    yend = [pos_problem[i][1] + 1500 for i in range(len(ts))]
    skeletons_small = []
    for i in range(len(ts)):
        skeletons_small.append(
            exp.skeletons[ts[i]][xbegin[i] : xend[i], ybegin[i] : yend[i]]
        )
    if ts[-1] < len(exp.skeletons) - 1:
        skeletons_small.append(
            exp.skeletons[ts[-1] + 1][xbegin[-1] : xend[-1], ybegin[-1] : yend[-1]]
        )
    node_smalls = []
    for i in range(len(ts)):
        node_smalls.append(
            [
                node
                for node in exp.nx_graph[ts[i]].nodes
                if (
                    xbegin[i] < poss[ts[i]][node][0] < xend[i]
                    and ybegin[i] < poss[ts[i]][node][1] < yend[i]
                    and exp.nx_graph[ts[i]].degree(node) >= 1
                )
            ]
        )
    kernel = np.ones((5, 5), np.uint8)
    skeletons_small_dilated = [
        cv2.dilate(skeleton.todense().astype(np.uint8), kernel, iterations=1)
        for skeleton in skeletons_small
    ]
    for tp1 in range(len(node_smalls)):
        plot_t_tp1(
            [node_interest],
            [node_interest],
            poss[ts[tp1]],
            poss[ts[tp1]],
            skeletons_small_dilated[tp1],
            skeletons_small_dilated[tp1],
            shift=(xbegin[tp1], ybegin[tp1]),
            save=f"Data/video_test/network_timestep_{tp1}",
            time=f"t={4*tp1}h",
        )
        clear_output(wait=True)
    if ts[-1] < len(exp.skeletons) - 1:
        plot_t_tp1(
            [],
            [],
            poss[ts[-1] + 1],
            poss[ts[-1] + 1],
            skeletons_small_dilated[-1],
            skeletons_small_dilated[-1],
            shift=(xbegin[-1], ybegin[-1]),
            save=f"Data/video_test/network_timestep_{len(node_smalls)}",
            time=f"t={len(node_smalls)}h",
        )
        clear_output(wait=True)
    images = []
    for t in range(len(node_smalls)):
        images.append(imageio.imread(f"Data/video_test/network_timestep_{t}.png"))
    if ts[-1] < len(exp.skeletons) - 1:
        images.append(
            imageio.imread(f"Data/video_test/network_timestep_{len(node_smalls)}.png")
        )
    imageio.mimsave(
        f"Data/video_test/anast_no_clean/{node_interest}movie_track.gif",
        images,
        duration=1,
    )
    plt.close("all")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
tip = choice(anastomose)
node_interest = tip
poss = exp.positions
ts = [t for t in range(len(poss)) if exp.get_node(tip).is_in(t)]
pos_problem = [poss[i][node_interest] for i in ts]
xbegin = [pos_problem[i][0] - 1500 for i in range(len(ts))]
ybegin = [pos_problem[i][1] - 1500 for i in range(len(ts))]
xend = [pos_problem[i][0] + 1500 for i in range(len(ts))]
yend = [pos_problem[i][1] + 1500 for i in range(len(ts))]
skeletons_small = []
for i in range(len(ts)):
    skeletons_small.append(
        exp.skeletons[ts[i]][xbegin[i] : xend[i], ybegin[i] : yend[i]]
    )
if ts[-1] < len(exp.skeletons) - 1:
    skeletons_small.append(
        exp.skeletons[ts[-1] + 1][xbegin[-1] : xend[-1], ybegin[-1] : yend[-1]]
    )
node_smalls = []
for i in range(len(ts)):
    node_smalls.append(
        [
            node
            for node in exp.nx_graph[ts[i]].nodes
            if (
                xbegin[i] < poss[ts[i]][node][0] < xend[i]
                and ybegin[i] < poss[ts[i]][node][1] < yend[i]
                and exp.nx_graph[ts[i]].degree(node) >= 1
            )
        ]
    )
kernel = np.ones((5, 5), np.uint8)
skeletons_small_dilated = [
    cv2.dilate(skeleton.todense().astype(np.uint8), kernel, iterations=1)
    for skeleton in skeletons_small
]
for tp1 in range(len(node_smalls)):
    plot_t_tp1(
        node_smalls[tp1],
        node_smalls[tp1],
        poss[ts[tp1]],
        poss[ts[tp1]],
        skeletons_small_dilated[tp1],
        skeletons_small_dilated[tp1],
        shift=(xbegin[tp1], ybegin[tp1]),
        save=f"Data/video_test/network_timestep_{tp1}",
        time=f"t={4*tp1}h",
    )
    clear_output(wait=True)
if ts[-1] < len(exp.skeletons) - 1:
    plot_t_tp1(
        [],
        [],
        poss[ts[-1] + 1],
        poss[ts[-1] + 1],
        skeletons_small_dilated[-1],
        skeletons_small_dilated[-1],
        shift=(xbegin[-1], ybegin[-1]),
        save=f"Data/video_test/network_timestep_{tp1}",
        time=f"t={len(node_smalls)}h",
    )
    clear_output(wait=True)
images = []
for t in range(len(node_smalls)):
    images.append(imageio.imread(f"Data/video_test/network_timestep_{t}.png"))
if ts[-1] < len(exp.skeletons) - 1:
    images.append(
        imageio.imread(f"Data/video_test/network_timestep_{len(node_smalls)}.png")
    )
imageio.mimsave(f"Data/video_test/{node_interest}movie_track.gif", images, duration=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [74]:
tip

4847

In [199]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("time (h)")
ax.set_ylabel("growth (mm/day)")
(spotting,) = ax.plot(
    np.array(range(len(tip_growth[tip]))) * 4,
    np.array(tip_growth[tip]) * 1.725 / 4 * 24 / 1000,
)
spotting.set_label(str(tip))
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
for i in range(10):
    tip = choice(stop)
    node_interest = tip
    poss = exp.positions
    ts = [t for t in range(len(poss)) if exp.get_node(tip).is_in(t)]
    pos_problem = [poss[i][node_interest] for i in ts]
    xbegin = [pos_problem[i][0] - 1500 for i in range(len(ts))]
    ybegin = [pos_problem[i][1] - 1500 for i in range(len(ts))]
    xend = [pos_problem[i][0] + 1500 for i in range(len(ts))]
    yend = [pos_problem[i][1] + 1500 for i in range(len(ts))]
    skeletons_small = []
    for i in range(len(ts)):
        skeletons_small.append(
            exp.skeletons[ts[i]][xbegin[i] : xend[i], ybegin[i] : yend[i]]
        )
    if ts[-1] < len(exp.skeletons) - 1:
        skeletons_small.append(
            exp.skeletons[ts[-1] + 1][xbegin[-1] : xend[-1], ybegin[-1] : yend[-1]]
        )
    node_smalls = []
    for i in range(len(ts)):
        node_smalls.append(
            [
                node
                for node in exp.nx_graph[ts[i]].nodes
                if (
                    xbegin[i] < poss[ts[i]][node][0] < xend[i]
                    and ybegin[i] < poss[ts[i]][node][1] < yend[i]
                    and exp.nx_graph[ts[i]].degree(node) >= 1
                )
            ]
        )
    kernel = np.ones((5, 5), np.uint8)
    skeletons_small_dilated = [
        cv2.dilate(skeleton.todense().astype(np.uint8), kernel, iterations=1)
        for skeleton in skeletons_small
    ]
    for tp1 in range(len(node_smalls)):
        plot_t_tp1(
            [node_interest],
            [node_interest],
            poss[ts[tp1]],
            poss[ts[tp1]],
            skeletons_small_dilated[tp1],
            skeletons_small_dilated[tp1],
            shift=(xbegin[tp1], ybegin[tp1]),
            save=f"Data/video_test/network_timestep_{tp1}",
            time=f"t={4*tp1}h",
        )
        clear_output(wait=True)
    if ts[-1] < len(exp.skeletons) - 1:
        plot_t_tp1(
            [],
            [],
            poss[ts[-1] + 1],
            poss[ts[-1] + 1],
            skeletons_small_dilated[-1],
            skeletons_small_dilated[-1],
            shift=(xbegin[-1], ybegin[-1]),
            save=f"Data/video_test/network_timestep_{len(node_smalls)}",
            time=f"t={len(node_smalls)}h",
        )
        clear_output(wait=True)
    images = []
    for t in range(len(node_smalls)):
        images.append(imageio.imread(f"Data/video_test/network_timestep_{t}.png"))
    if ts[-1] < len(exp.skeletons) - 1:
        images.append(
            imageio.imread(f"Data/video_test/network_timestep_{len(node_smalls)}.png")
        )
    imageio.mimsave(
        f"Data/video_test/stop_no_clean/{node_interest}movie_track.gif",
        images,
        duration=1,
    )
    plt.close("all")
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.set_xlabel('time (h)')
# ax.set_ylabel('growth (mm/day)')
# spotting,=ax.plot(np.array(range(len(tip_growth[tip])))*4,np.array(tip_growth[tip])*1.725/4*24/1000)
# spotting.set_label(str(tip))
# ax.legend()
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
for i in range(10):
    tip = choice(stop_with_long_growth)
    node_interest = tip
    poss = exp.positions
    ts = [t for t in range(len(poss)) if exp.get_node(tip).is_in(t)]
    pos_problem = [poss[i][node_interest] for i in ts]
    xbegin = [pos_problem[i][0] - 1500 for i in range(len(ts))]
    ybegin = [pos_problem[i][1] - 1500 for i in range(len(ts))]
    xend = [pos_problem[i][0] + 1500 for i in range(len(ts))]
    yend = [pos_problem[i][1] + 1500 for i in range(len(ts))]
    skeletons_small = []
    for i in range(len(ts)):
        skeletons_small.append(
            exp.skeletons[ts[i]][xbegin[i] : xend[i], ybegin[i] : yend[i]]
        )
    if ts[-1] < len(exp.skeletons) - 1:
        skeletons_small.append(
            exp.skeletons[ts[-1] + 1][xbegin[-1] : xend[-1], ybegin[-1] : yend[-1]]
        )
    node_smalls = []
    for i in range(len(ts)):
        node_smalls.append(
            [
                node
                for node in exp.nx_graph[ts[i]].nodes
                if (
                    xbegin[i] < poss[ts[i]][node][0] < xend[i]
                    and ybegin[i] < poss[ts[i]][node][1] < yend[i]
                    and exp.nx_graph[ts[i]].degree(node) >= 1
                )
            ]
        )
    kernel = np.ones((5, 5), np.uint8)
    skeletons_small_dilated = [
        cv2.dilate(skeleton.todense().astype(np.uint8), kernel, iterations=1)
        for skeleton in skeletons_small
    ]
    for tp1 in range(len(node_smalls)):
        plot_t_tp1(
            [node_interest],
            [node_interest],
            poss[ts[tp1]],
            poss[ts[tp1]],
            skeletons_small_dilated[tp1],
            skeletons_small_dilated[tp1],
            shift=(xbegin[tp1], ybegin[tp1]),
            save=f"Data/video_test/network_timestep_{tp1}",
            time=f"t={4*tp1}h",
        )
        clear_output(wait=True)
    if ts[-1] < len(exp.skeletons) - 1:
        plot_t_tp1(
            [],
            [],
            poss[ts[-1] + 1],
            poss[ts[-1] + 1],
            skeletons_small_dilated[-1],
            skeletons_small_dilated[-1],
            shift=(xbegin[-1], ybegin[-1]),
            save=f"Data/video_test/network_timestep_{tp1}",
            time=f"t={len(node_smalls)}h",
        )
        clear_output(wait=True)
    images = []
    for t in range(len(node_smalls)):
        images.append(imageio.imread(f"Data/video_test/network_timestep_{t}.png"))
    if ts[-1] < len(exp.skeletons) - 1:
        images.append(
            imageio.imread(f"Data/video_test/network_timestep_{len(node_smalls)}.png")
        )
    imageio.mimsave(
        f"Data/video_test/stop_long/{node_interest}movie_track.gif", images, duration=1
    )
    plt.close("all")
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.set_xlabel('time (h)')
# ax.set_ylabel('growth (mm/day)')
# spotting,=ax.plot(np.array(range(len(tip_growth[tip])))*4,np.array(tip_growth[tip])*1.725/4*24/1000)
# spotting.set_label(str(tip))
# ax.legend()
# plt.show()

In [209]:
30 * 1.725 / 4 * 24 / 1000

0.3105

In [206]:
choice(stop)

1968

In [ ]:
exp.save()

In [37]:
exp2 = Experiment(13)
exp2.copy(exp)

In [28]:
L = pinpoint_anastomosis(exp, t)[1]

In [57]:
node = 64
node_obj = ex
p2.get_node(node)
node_obj.pos(t)

array([12970, 18220])

In [52]:
for node in L:
    node = 830
    node_obj = exp2.get_node(node)
    imtm1, posstm1 = exp2.find_image_pos(
        node_obj.pos(t)[0], node_obj.pos(t)[1], t, local=False
    )
    imt, posst = exp2.find_image_pos(
        node_obj.pos(t)[0], node_obj.pos(t)[1], t + 1, local=False
    )
    i = 0
    plot_t_tp1(
        [node], [], {node: (posstm1[1][i], posstm1[0][i])}, None, imtm1[i], imt[i]
    )
    break

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
t = 0
exp2.plot([t, t + 1], [pinpoint_anastomosis(exp2, t)[1]] * 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
4725 * 5, 2908 * 5

(23625, 14540)

In [18]:
anas_result = [pinpoint_anastomosis(exp2, i) for i in range(len(dates) - 1)]
anastomosiss = [ana_result[0] for ana_result in anas_result]
origins = [ana_result[1] for ana_result in anas_result]
number_an = [ana_result[2] for ana_result in anas_result]

In [22]:
degree4_nodes = [
    [node for node in nx_graph.nodes if nx_graph.degree(node) == 4]
    for nx_graph in exp2.nx_graph
]
numberdegree4 = [len(liste) for liste in degree4_nodes]
number_anastomosis_theory = [
    2 + len(nx_graph.edges) - len(nx_graph.nodes) - numberdegree4[i]
    for i, nx_graph in enumerate(exp2.nx_graph)
]
number_anastomosis_spot = [number_anastomosis_theory[0]] + [
    number_anastomosis_theory[0] + np.cumsum(number_an)[i]
    for i in range(len(anastomosiss))
]
per_frame_anastomosis_theory = [
    number_anastomosis_theory[i + 1] - number_anastomosis_theory[i]
    for i in range(len(number_anastomosis_theory) - 1)
]
per_frame_anastomosis_spot = [
    number_anastomosis_spot[i + 1] - number_anastomosis_spot[i]
    for i in range(len(number_anastomosis_theory) - 1)
]

In [25]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("time (h)")
ax.set_ylabel("number anastomosis per frame")
(theory,) = ax.plot(per_frame_anastomosis_theory[:-6])
theory.set_label("theory")
(spotting,) = ax.plot(per_frame_anastomosis_spot[:-6])
spotting.set_label("frame spotting")
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("time (h)")
ax.set_ylabel("total number of anastomosis")
(theory,) = ax.plot(number_anastomosis_theory[:-6])
theory.set_label("theory")
(spotting,) = ax.plot(number_anastomosis_spot[:-6])
spotting.set_label("frame spotting")
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
plt.close("all")